<a href="https://colab.research.google.com/github/abuelnasr0/Semantic-Search-project/blob/youtube-captions/Notebooks/data_preprocessing/youtube_captions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook, we will get a playlist ID and use the youtube API to get and 
save the captions of the videos in that playlist

In [ ]:
!pip install youtube_transcript_api

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi
import googleapiclient.discovery 
import numpy as np
import time

In [ ]:
DEVELOPER_KEY = ""
YOUTUBE_API_SERVICE_NAME = ""
YOUTUBE_API_VERSION = ""

In [ ]:
def video_to_captions(video_id):
    done = 0
    captions = {}
    try:
        srt = YouTubeTranscriptApi.get_transcript(video_id)
        captions["video_id"] = video_id
        captions["video_captions"] = srt
        done = 1
    except:
        done = 0
        
    return [captions] , done 
    
def playlist_to_captions(playlist_id):
    
    youtube = googleapiclient.discovery.build(YOUTUBE_API_SERVICE_NAME,
                    YOUTUBE_API_VERSION,
                    developerKey=DEVELOPER_KEY)
    
    res = youtube.playlistItems().list(part="snippet",
                                        playlistId=playlist_id,
                                        maxResults="50"
                                        ).execute()

    nextPageToken = res.get('nextPageToken')
    
    while ('nextPageToken' in res):
        nextPage = youtube.playlistItems().list(
        part="snippet",
        playlistId=playlist_id,
        maxResults="50",
        pageToken=nextPageToken
        ).execute()
        res['items'] = res['items'] + nextPage['items']

        if 'nextPageToken' not in nextPage:
            res.pop('nextPageToken', None)
        else:
            nextPageToken = nextPage['nextPageToken']

    captions = []
    # [{'video_id': 'nfWlot6h_JM', 'video_title': 'title'},  ]

    for v in res["items"]:
        dic = {}
        vid = v['snippet']["resourceId"]['videoId']
        dic["video_id"] = vid
        caps , done = video_to_captions(vid)        
        if(done):
            dic["video_captions"] = caps[0]["video_captions"]
        else:
            continue
        captions.append(dic)
        
    return captions

In [ ]:
def captions_processing(caps):
    for cap in caps:
        it = 0 
        length = len(cap['video_captions'])
        new = []
        i = 0
        while(i < length): 

            cap['video_captions'][i]['text'] =  cap['video_captions'][i]['text'].replace("\n"," ")

            if cap['video_captions'][i]["text"][-1] == '.' or cap['video_captions'][i]["text"][-1] == "?":
                dic={}
                dic['text'] = cap['video_captions'][it]['text']
                dic['start'] = cap['video_captions'][it]['start']
                dic['duration'] = cap['video_captions'][it]['duration']
                for j in range(it+1,i+1): 
                    dic['text'] += " " + cap['video_captions'][j]['text']
                    dic['duration'] += cap['video_captions'][j]['duration']

                cap['video_captions'][it] = dic 

                for j in range(it+1,i+1): 
                    cap['video_captions'].pop(it+1)


                i = it
                it +=1
                length = len(cap['video_captions'])
                if i >= length :
                    break
            i+=1     
    return caps

In [ ]:
playlist_id = "PLUl4u3cNGP619EG1wp0kT-7rDE_Az5TNd"
captions = playlist_to_captions(playlist_id)
processed = captions_processing(captions)

In [ ]:
playlist_captions = {
    "playlist_id" : playlist_id,
    "videos" : processed
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json
json_object = json.dumps(playlist_captions, indent=4)
with open(f"/content/drive/MyDrive/Graduation_project/Data/Youtube_captions/Captions/{playlist_id}.json", "w") as outfile:
    outfile.write(json_object)